# Setup

This is following [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) for pandas 1.4.3

In [1]:
import numpy as np
import pandas as pd

# Object creation

In [2]:
# Series - pass a list to pd.Series
s1 = pd.Series([1, 2, 3, np.nan, 5])

In [3]:
s1

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
dtype: float64

In [4]:
# create an index to use in the dataframe
# default frequency is D (day), so creates a DatetimeIndex of 10 days
dates = pd.date_range('20220619', periods=10)
dates

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

just in case you are wondering, here's [all the "offset aliases"](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases) for Datetime

In [5]:
# Dataframe - pass a Numpy array
# create a 10 row, 4 col random number array, index by dates, give some column names
df = pd.DataFrame(np.random.randn(10, 4), index=dates, columns=list('ABCD'))
df

,A,B,C,D
2022-06-19,1.525287,-1.583237,0.131429,0.999250
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889
2022-06-21,-0.024668,-1.885221,0.434181,0.383347
2022-06-22,0.046389,0.103630,-0.387595,-0.577285
2022-06-23,0.681873,1.155926,1.646095,-0.302723
2022-06-24,0.252842,-0.594838,-0.827767,0.166141
2022-06-25,-0.536672,1.097415,0.257771,0.429608
2022-06-26,0.259723,1.056366,1.013331,1.317313
2022-06-27,-0.912890,0.185351,-1.087552,0.353240
2022-06-28,0.495321,0.684865,-0.746881,1.122277


In [6]:
# Create a dataframe by passing a dictionary of objects
# where each object can be converted into a series-like structure
df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20220619"),
        "C": pd.Series(1, index=list(range(6)), dtype="float32"),
        "D": np.array([3]*6, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train", "test", "train"]),
        "F": list("foofoo"),
        "G": "foo"
    }
)

df2

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


In [7]:
# the datatype of each of the columns would be different
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
G            object
dtype: object

# Viewing Data

In [8]:
# top of the dataframe
df.head()

,A,B,C,D
2022-06-19,1.525287,-1.583237,0.131429,0.999250
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889
2022-06-21,-0.024668,-1.885221,0.434181,0.383347
2022-06-22,0.046389,0.103630,-0.387595,-0.577285
2022-06-23,0.681873,1.155926,1.646095,-0.302723


In [9]:
# bottom 3 records of the dataframe
df.tail(3)

,A,B,C,D
2022-06-26,0.259723,1.056366,1.013331,1.317313
2022-06-27,-0.912890,0.185351,-1.087552,0.353240
2022-06-28,0.495321,0.684865,-0.746881,1.122277


In [10]:
# index of the df
df.index

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

In [11]:
# columns of the df
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [12]:
# .to_numpy gives a NumPy representation of the dataframe
# this is expensive if all columns are of different data type
df.to_numpy()

array([[ 1.52528688, -1.58323674,  0.13142861,  0.99925032],
       [-1.1393869 , -2.54909915, -0.52391719, -1.21588882],
       [-0.02466816, -1.88522105,  0.43418085,  0.38334731],
       [ 0.04638908,  0.10363003, -0.38759501, -0.5772851 ],
       [ 0.68187282,  1.15592606,  1.64609544, -0.30272334],
       [ 0.25284227, -0.59483784, -0.82776689,  0.1661415 ],
       [-0.53667173,  1.09741484,  0.25777104,  0.42960782],
       [ 0.25972289,  1.05636612,  1.01333052,  1.31731292],
       [-0.91289047,  0.18535064, -1.08755238,  0.35324031],
       [ 0.49532123,  0.68486499, -0.74688147,  1.12227699]])

In [13]:
df2.to_numpy()

array([[1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo']], dtype=object)

In [14]:
# quick summary stats
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.064782,-0.232884,-0.009091,0.267528
std,0.785179,1.355672,0.872951,0.793361
min,-1.139387,-2.549099,-1.087552,-1.215889
25%,-0.408671,-1.336137,-0.691140,-0.185507
50%,0.149616,0.144490,-0.128083,0.368294
75%,0.436422,0.963491,0.390078,0.856840
max,1.525287,1.155926,1.646095,1.317313


1. **count** = Count number of non-NA/null observations
1. **max** = Maximum of the values in the object
1. **min** = Minimum of the values in the object
1. **mean** = Mean of the values
1. **std** = Standard deviation of the observations
1. **25%** = Default lower percentile
1. **50%** = 50 percentile - same as the median 
1. **75%** = Default upper percentile

In [15]:
# change the percentiles
df.describe(percentiles=[.1, .5, .9])

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.064782,-0.232884,-0.009091,0.267528
std,0.785179,1.355672,0.872951,0.793361
min,-1.139387,-2.549099,-1.087552,-1.215889
10%,-0.935540,-1.951609,-0.853745,-0.641145
50%,0.149616,0.144490,-0.128083,0.368294
90%,0.766214,1.103266,1.076607,1.141781
max,1.525287,1.155926,1.646095,1.317313


In [16]:
# E, F, G in d2 are not numeric, so do not come up in describe()
df2.describe()

,A,C,D
count,6.0,6.0,6.0
mean,1.0,1.0,3.0
std,0.0,0.0,0.0
min,1.0,1.0,3.0
25%,1.0,1.0,3.0
50%,1.0,1.0,3.0
75%,1.0,1.0,3.0
max,1.0,1.0,3.0


In [17]:
# transpose the data
# turn rows to columns and vice versa
df.T

,2022-06-19,2022-06-20,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28
A,1.525287,-1.139387,-0.024668,0.046389,0.681873,0.252842,-0.536672,0.259723,-0.912890,0.495321
B,-1.583237,-2.549099,-1.885221,0.103630,1.155926,-0.594838,1.097415,1.056366,0.185351,0.684865
C,0.131429,-0.523917,0.434181,-0.387595,1.646095,-0.827767,0.257771,1.013331,-1.087552,-0.746881
D,0.999250,-1.215889,0.383347,-0.577285,-0.302723,0.166141,0.429608,1.317313,0.353240,1.122277


### [Axis in pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_axis.html?highlight=set_ax#pandas.DataFrame.set_axis)

```axis{0 or ‘index’, 1 or ‘columns’}```

Always remember that when you specify:
* ```axis=0``` or ```axis='index'``` you indicate that the operation should be along the _index_ (aka across the rows). More often than not indexes will go from top to bottom (vertically), but sometimes they may not (for e.g. if you use columns as index or when you have hierarchical or multi-index data).
* ```axis=1``` or ```axis='columns'``` indicates the operation is along the columns

When in doubt use the explicit version ```'index'``` or ```'columns'``` instead of ```0``` or ```1```

In [18]:
# sort along the axis - 1 = horizontal
df.sort_index(axis=1, ascending=False)

,D,C,B,A
2022-06-19,0.999250,0.131429,-1.583237,1.525287
2022-06-20,-1.215889,-0.523917,-2.549099,-1.139387
2022-06-21,0.383347,0.434181,-1.885221,-0.024668
2022-06-22,-0.577285,-0.387595,0.103630,0.046389
2022-06-23,-0.302723,1.646095,1.155926,0.681873
2022-06-24,0.166141,-0.827767,-0.594838,0.252842
2022-06-25,0.429608,0.257771,1.097415,-0.536672
2022-06-26,1.317313,1.013331,1.056366,0.259723
2022-06-27,0.353240,-1.087552,0.185351,-0.912890
2022-06-28,1.122277,-0.746881,0.684865,0.495321


In [19]:
# sort along the axis - 0 = vertical
df.sort_index(axis=0, ascending=False)

,A,B,C,D
2022-06-28,0.495321,0.684865,-0.746881,1.122277
2022-06-27,-0.912890,0.185351,-1.087552,0.353240
2022-06-26,0.259723,1.056366,1.013331,1.317313
2022-06-25,-0.536672,1.097415,0.257771,0.429608
2022-06-24,0.252842,-0.594838,-0.827767,0.166141
2022-06-23,0.681873,1.155926,1.646095,-0.302723
2022-06-22,0.046389,0.103630,-0.387595,-0.577285
2022-06-21,-0.024668,-1.885221,0.434181,0.383347
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889
2022-06-19,1.525287,-1.583237,0.131429,0.999250


In [20]:
# sort ascending by values in a column
df.sort_values(by="A")

,A,B,C,D
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889
2022-06-27,-0.912890,0.185351,-1.087552,0.353240
2022-06-25,-0.536672,1.097415,0.257771,0.429608
2022-06-21,-0.024668,-1.885221,0.434181,0.383347
2022-06-22,0.046389,0.103630,-0.387595,-0.577285
2022-06-24,0.252842,-0.594838,-0.827767,0.166141
2022-06-26,0.259723,1.056366,1.013331,1.317313
2022-06-28,0.495321,0.684865,-0.746881,1.122277
2022-06-23,0.681873,1.155926,1.646095,-0.302723
2022-06-19,1.525287,-1.583237,0.131429,0.999250


In [21]:
# sort by non-numerical values
df2.sort_values(by="F", ascending=False)

,A,B,C,D,E,F,G
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo


In [22]:
# sort by two or more columns
df2.sort_values(by=["F", "E"])

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


# Selection

For production prefer the following instead of other data access methods (typical python methods like ["col"] or [a:b] slices etc.):

```.at, .iat, .loc and .iloc.```

## Getting

In [23]:
# selecting a single column returns a Series object
df["A"]

2022-06-19    1.525287
2022-06-20   -1.139387
2022-06-21   -0.024668
2022-06-22    0.046389
2022-06-23    0.681873
2022-06-24    0.252842
2022-06-25   -0.536672
2022-06-26    0.259723
2022-06-27   -0.912890
2022-06-28    0.495321
Freq: D, Name: A, dtype: float64

In [24]:
# selecting a slice
df[1:5]

,A,B,C,D
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889
2022-06-21,-0.024668,-1.885221,0.434181,0.383347
2022-06-22,0.046389,0.103630,-0.387595,-0.577285
2022-06-23,0.681873,1.155926,1.646095,-0.302723


## Selection by label

In [25]:
# selecting based on a label
df.loc[dates[0]]

A    1.525287
B   -1.583237
C    0.131429
D    0.999250
Name: 2022-06-19 00:00:00, dtype: float64

In [26]:
# select on a multi-axis by lable
# I honestly do not know what the comment above means...help!
df.loc[:, ["A", "B"]]

,A,B
2022-06-19,1.525287,-1.583237
2022-06-20,-1.139387,-2.549099
2022-06-21,-0.024668,-1.885221
2022-06-22,0.046389,0.103630
2022-06-23,0.681873,1.155926
2022-06-24,0.252842,-0.594838
2022-06-25,-0.536672,1.097415
2022-06-26,0.259723,1.056366
2022-06-27,-0.912890,0.185351
2022-06-28,0.495321,0.684865


In [27]:
# specific index value results in reduction of dimensions
res = df.loc["2022-06-20"]
print(res)
print("res.shape = ", res.shape, " vs. df.shape = ", df.shape)

A   -1.139387
B   -2.549099
C   -0.523917
D   -1.215889
Name: 2022-06-20 00:00:00, dtype: float64
res.shape =  (4,)  vs. df.shape =  (10, 4)


In [28]:
# get to a specific scalar:
#
# method one
df.loc[dates[0], "A"]

1.5252868758903537

In [29]:
#
# method two (slightly faster than method one)
df.at[dates[0], "A"]

1.5252868758903537

## Selection by position

In [30]:
df.iloc[2]

A   -0.024668
B   -1.885221
C    0.434181
D    0.383347
Name: 2022-06-21 00:00:00, dtype: float64

In [31]:
# slices - similar to NumPy / Python - [row:slice, col:slice]
df.iloc[1:5, 0:2]

,A,B
2022-06-20,-1.139387,-2.549099
2022-06-21,-0.024668,-1.885221
2022-06-22,0.046389,0.103630
2022-06-23,0.681873,1.155926


In [32]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
df.iloc[[0, 1, 2, 6], [0, 2]]

,A,C
2022-06-19,1.525287,0.131429
2022-06-20,-1.139387,-0.523917
2022-06-21,-0.024668,0.434181
2022-06-25,-0.536672,0.257771


In [33]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
# change the order of columns, repeact a column
df.iloc[[0, 1, 2, 6], [2, 1, 0, 2]]

,C,B,A,C
2022-06-19,0.131429,-1.583237,1.525287,0.131429
2022-06-20,-0.523917,-2.549099,-1.139387,-0.523917
2022-06-21,0.434181,-1.885221,-0.024668,0.434181
2022-06-25,0.257771,1.097415,-0.536672,0.257771


In [34]:
# slice rows explicitly, keep all columns
df.iloc[[1, 2], :]

,A,B,C,D
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889
2022-06-21,-0.024668,-1.885221,0.434181,0.383347


In [35]:
# slice columns, keep all rows
df.iloc[:, [2, 3]]

,C,D
2022-06-19,0.131429,0.999250
2022-06-20,-0.523917,-1.215889
2022-06-21,0.434181,0.383347
2022-06-22,-0.387595,-0.577285
2022-06-23,1.646095,-0.302723
2022-06-24,-0.827767,0.166141
2022-06-25,0.257771,0.429608
2022-06-26,1.013331,1.317313
2022-06-27,-1.087552,0.353240
2022-06-28,-0.746881,1.122277


In [36]:
# everything, because you can
df.iloc[:, :]

,A,B,C,D
2022-06-19,1.525287,-1.583237,0.131429,0.999250
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889
2022-06-21,-0.024668,-1.885221,0.434181,0.383347
2022-06-22,0.046389,0.103630,-0.387595,-0.577285
2022-06-23,0.681873,1.155926,1.646095,-0.302723
2022-06-24,0.252842,-0.594838,-0.827767,0.166141
2022-06-25,-0.536672,1.097415,0.257771,0.429608
2022-06-26,0.259723,1.056366,1.013331,1.317313
2022-06-27,-0.912890,0.185351,-1.087552,0.353240
2022-06-28,0.495321,0.684865,-0.746881,1.122277


In [37]:
# get to a scalar (2 methods, just like before)
#
# method one: use iloc
df.iloc[1, 2]

-0.5239171931040062

In [38]:
#
# method two: use iat
df.iat[1, 2]

-0.5239171931040062

## Boolean Indexing

In [39]:
# use a value found in a single col to get data
df[df["A"] > 0]

,A,B,C,D
2022-06-19,1.525287,-1.583237,0.131429,0.999250
2022-06-22,0.046389,0.103630,-0.387595,-0.577285
2022-06-23,0.681873,1.155926,1.646095,-0.302723
2022-06-24,0.252842,-0.594838,-0.827767,0.166141
2022-06-26,0.259723,1.056366,1.013331,1.317313
2022-06-28,0.495321,0.684865,-0.746881,1.122277


In [40]:
# boolean across the entire DF - vals that don't match go NaN
df[df > 0]

,A,B,C,D
2022-06-19,1.525287,NaN,0.131429,0.999250
2022-06-20,NaN,NaN,NaN,NaN
2022-06-21,NaN,NaN,0.434181,0.383347
2022-06-22,0.046389,0.103630,NaN,NaN
2022-06-23,0.681873,1.155926,1.646095,NaN
2022-06-24,0.252842,NaN,NaN,0.166141
2022-06-25,NaN,1.097415,0.257771,0.429608
2022-06-26,0.259723,1.056366,1.013331,1.317313
2022-06-27,NaN,0.185351,NaN,0.353240
2022-06-28,0.495321,0.684865,NaN,1.122277


In [41]:
# add another column
df11 = df.copy()
df11["E"] = ["one", "two", "three", "four",
             "two", "five", "one", "two", "three", "four"]
df11

,A,B,C,D,E
2022-06-19,1.525287,-1.583237,0.131429,0.999250,one
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889,two
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,three
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,four
2022-06-23,0.681873,1.155926,1.646095,-0.302723,two
2022-06-24,0.252842,-0.594838,-0.827767,0.166141,five
2022-06-25,-0.536672,1.097415,0.257771,0.429608,one
2022-06-26,0.259723,1.056366,1.013331,1.317313,two
2022-06-27,-0.912890,0.185351,-1.087552,0.353240,three
2022-06-28,0.495321,0.684865,-0.746881,1.122277,four


In [42]:
# the isin() query - basically the in clause
df11[df11["E"].isin(["two", "five"])]

,A,B,C,D,E
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889,two
2022-06-23,0.681873,1.155926,1.646095,-0.302723,two
2022-06-24,0.252842,-0.594838,-0.827767,0.166141,five
2022-06-26,0.259723,1.056366,1.013331,1.317313,two


## Setting values

In [43]:
# matching indexes auto-aligns values
s1 = pd.Series(range(11, 21), index=pd.date_range('20220619', periods=10))
s1

2022-06-19    11
2022-06-20    12
2022-06-21    13
2022-06-22    14
2022-06-23    15
2022-06-24    16
2022-06-25    17
2022-06-26    18
2022-06-27    19
2022-06-28    20
Freq: D, dtype: int64

In [44]:
df["F"] = s1
df

,A,B,C,D,F
2022-06-19,1.525287,-1.583237,0.131429,0.999250,11
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889,12
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14
2022-06-23,0.681873,1.155926,1.646095,-0.302723,15
2022-06-24,0.252842,-0.594838,-0.827767,0.166141,16
2022-06-25,-0.536672,1.097415,0.257771,0.429608,17
2022-06-26,0.259723,1.056366,1.013331,1.317313,18
2022-06-27,-0.912890,0.185351,-1.087552,0.353240,19
2022-06-28,0.495321,0.684865,-0.746881,1.122277,20


In [45]:
# setting values by label and position
# first let's make a quick copy
df12 = df.copy()
df12

,A,B,C,D,F
2022-06-19,1.525287,-1.583237,0.131429,0.999250,11
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889,12
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14
2022-06-23,0.681873,1.155926,1.646095,-0.302723,15
2022-06-24,0.252842,-0.594838,-0.827767,0.166141,16
2022-06-25,-0.536672,1.097415,0.257771,0.429608,17
2022-06-26,0.259723,1.056366,1.013331,1.317313,18
2022-06-27,-0.912890,0.185351,-1.087552,0.353240,19
2022-06-28,0.495321,0.684865,-0.746881,1.122277,20


In [46]:
# set by label
df12.at[dates[0], "A"] = 0
# set by position
df12.iat[0, 1] = 0
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,0.131429,0.999250,11
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889,12
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14
2022-06-23,0.681873,1.155926,1.646095,-0.302723,15
2022-06-24,0.252842,-0.594838,-0.827767,0.166141,16
2022-06-25,-0.536672,1.097415,0.257771,0.429608,17
2022-06-26,0.259723,1.056366,1.013331,1.317313,18
2022-06-27,-0.912890,0.185351,-1.087552,0.353240,19
2022-06-28,0.495321,0.684865,-0.746881,1.122277,20


In [47]:
# kinda bigger replacement
df12.loc[:, "D"] = np.array([5]*len(df))
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,0.131429,5,11
2022-06-20,-1.139387,-2.549099,-0.523917,5,12
2022-06-21,-0.024668,-1.885221,0.434181,5,13
2022-06-22,0.046389,0.103630,-0.387595,5,14
2022-06-23,0.681873,1.155926,1.646095,5,15
2022-06-24,0.252842,-0.594838,-0.827767,5,16
2022-06-25,-0.536672,1.097415,0.257771,5,17
2022-06-26,0.259723,1.056366,1.013331,5,18
2022-06-27,-0.912890,0.185351,-1.087552,5,19
2022-06-28,0.495321,0.684865,-0.746881,5,20


In [48]:
# setting values using a boolean selection (aka where clause)
df12[df12 > 0] = -df12
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,-0.131429,-5,-11
2022-06-20,-1.139387,-2.549099,-0.523917,-5,-12
2022-06-21,-0.024668,-1.885221,-0.434181,-5,-13
2022-06-22,-0.046389,-0.103630,-0.387595,-5,-14
2022-06-23,-0.681873,-1.155926,-1.646095,-5,-15
2022-06-24,-0.252842,-0.594838,-0.827767,-5,-16
2022-06-25,-0.536672,-1.097415,-0.257771,-5,-17
2022-06-26,-0.259723,-1.056366,-1.013331,-5,-18
2022-06-27,-0.912890,-0.185351,-1.087552,-5,-19
2022-06-28,-0.495321,-0.684865,-0.746881,-5,-20


# Missing Data

### _reindex_
change/add/delete index on a specified axis, returns a new dataframe

In [49]:
df13 = df.reindex(index=dates[0:4], columns=list(df.columns)+["G"])
df13

,A,B,C,D,F,G
2022-06-19,1.525287,-1.583237,0.131429,0.999250,11,NaN
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889,12,NaN
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13,NaN
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14,NaN


### handling missing data
1. _```np.nan```_
1. _```pandas.isna()```_
1. ```df.dropna()```
1. ```df.fillna()```

In [50]:
# missing data in pandas is np.nan
df13.iat[1, 0] = np.nan
df13.iloc[1:, 5] = np.random.randint(1)
df13

,A,B,C,D,F,G
2022-06-19,1.525287,-1.583237,0.131429,0.999250,11,NaN
2022-06-20,NaN,-2.549099,-0.523917,-1.215889,12,0.0
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13,0.0
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14,0.0


In [51]:
# get a boolean mask where values are NaN
df131 = pd.isna(df13)
df131

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [52]:
# or just
pd.isna(df13)

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [53]:
# the original is still there
df13

,A,B,C,D,F,G
2022-06-19,1.525287,-1.583237,0.131429,0.999250,11,NaN
2022-06-20,NaN,-2.549099,-0.523917,-1.215889,12,0.0
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13,0.0
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14,0.0


In [54]:
# we are going to drop / replace values now, let's make a couple of copies of the dataframe
df132 = df13.copy()
df132

,A,B,C,D,F,G
2022-06-19,1.525287,-1.583237,0.131429,0.999250,11,NaN
2022-06-20,NaN,-2.549099,-0.523917,-1.215889,12,0.0
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13,0.0
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14,0.0


In [55]:
# drop rows/columns that have missing data
# by default it returns a new dataframe, you may want to specify inplace=True for modifying current dataframe:
df_no_na = df132.dropna(how="any")
# how=‘any’ : If any NA values are present, drop that row or column.
# how=‘all’ : If all values are NA, drop that row or column.

In [56]:
# all rows/cols with missing data stripped
df_no_na

,A,B,C,D,F,G
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13,0.0
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14,0.0


In [57]:
# original still intact
df132

,A,B,C,D,F,G
2022-06-19,1.525287,-1.583237,0.131429,0.999250,11,NaN
2022-06-20,NaN,-2.549099,-0.523917,-1.215889,12,0.0
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13,0.0
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14,0.0


In [58]:
# drop missing data from original
df132.dropna(how="any", inplace=True)
df132

,A,B,C,D,F,G
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13,0.0
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14,0.0


In [59]:
# fill missing data
df133 = df13.copy()
df133.fillna(np.pi*1000)

,A,B,C,D,F,G
2022-06-19,1.525287,-1.583237,0.131429,0.999250,11,3141.592654
2022-06-20,3141.592654,-2.549099,-0.523917,-1.215889,12,0.000000
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13,0.000000
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14,0.000000


# Align and Join

There needs to be a bigger notebook for this topic.
You need to know there's "joins" in Pandas just like in the SQL world, 
like join and left join and right join and inner and outer and all that...
Here we'll explore these in the context of the align function - 'aligning' the indexes of two dataframes.
Later we'll see these again in the context of _merging_ two data frames.

## Create two datasets with _mismatched_ indexes

In [60]:
# date range indexes
idx1 = pd.date_range('2022-01-01',periods = 10)
# 2022-01-01', '2022-01-02' don't exist in idx2
# '2022-01-11', '2022-01-12' don't exist in idx1
idx2 = pd.date_range('2022-01-03',periods = 10)

In [61]:
# dataframes from indexes
d1 = pd.DataFrame(index = idx1, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

d2 = pd.DataFrame(index = idx2, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

## Quick Aside: Rendering two dataframes side-by-side

The trick of rendering 2 dataframes side-by-side was from [this](https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side) stackoverflow question.

In [62]:
# we gon need to display both data frames side by side, so...
from IPython.display import display_html 
def render_df_side_by_side(a, b, a_title = "", b_title = ""):
    a_styler = a.style.set_table_attributes("style='display:inline'").set_caption(a_title)
    b_styler = b.style.set_table_attributes("style='display:inline'").set_caption(b_title)
    display_html(a_styler._repr_html_()+b_styler._repr_html_(), raw=True)

In [63]:
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.821870,13
2022-01-02 00:00:00,0.161661,18
2022-01-03 00:00:00,0.515014,7
2022-01-04 00:00:00,0.854069,16
2022-01-05 00:00:00,0.346648,9
2022-01-06 00:00:00,0.835336,1
2022-01-07 00:00:00,0.326867,7
2022-01-08 00:00:00,0.756280,23
2022-01-09 00:00:00,0.828364,1
2022-01-10 00:00:00,0.615816,17


Table rendering/styling options is [a bigger discussion](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#) to be tackled later.

## .add, .sub, .mul, .div etc.

flexible wrappers (```add, sub, mul, div, mod, pow```) to arithmetic operators: ```+, -, *, /, //, %, **```

In [64]:
# the +, -, *, /, //, %, ** operations align indexes to create a UNION of indexes
d3 = d1+d2
d4 = d1.sub(d2) #equivalent to d1-d2
d5 = d1*d2
d6 = d1.div(d2)

In [65]:
# indexs not present in the other dataframe get a NaN
d3.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.672436,24.000000
2022-01-04 00:00:00,0.854795,24.000000
2022-01-05 00:00:00,0.624787,24.000000
2022-01-06 00:00:00,1.172271,9.000000
2022-01-07 00:00:00,0.798002,11.000000
2022-01-08 00:00:00,1.037257,43.000000
2022-01-09 00:00:00,1.310482,11.000000
2022-01-10 00:00:00,1.387156,26.000000


In [66]:
# sub
d4.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.357592,-10.000000
2022-01-04 00:00:00,0.853342,8.000000
2022-01-05 00:00:00,0.068509,-6.000000
2022-01-06 00:00:00,0.498401,-7.000000
2022-01-07 00:00:00,-0.144269,3.000000
2022-01-08 00:00:00,0.475304,3.000000
2022-01-09 00:00:00,0.346247,-9.000000
2022-01-10 00:00:00,-0.155524,8.000000


In [67]:
# mul
d5.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.081074,119.000000
2022-01-04 00:00:00,0.000620,128.000000
2022-01-05 00:00:00,0.096416,135.000000
2022-01-06 00:00:00,0.281454,8.000000
2022-01-07 00:00:00,0.153999,28.000000
2022-01-08 00:00:00,0.212497,460.000000
2022-01-09 00:00:00,0.399369,10.000000
2022-01-10 00:00:00,0.475004,153.000000


In [68]:
# div
d6.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,3.271557,0.411765
2022-01-04 00:00:00,1175.991494,2.000000
2022-01-05 00:00:00,1.246312,0.600000
2022-01-06 00:00:00,2.479218,0.125000
2022-01-07 00:00:00,0.693785,1.750000
2022-01-08 00:00:00,2.691612,1.150000
2022-01-09 00:00:00,1.718180,0.100000
2022-01-10 00:00:00,0.798371,1.888889


### removing the NaN values from the resultant dataframe

In [69]:
# the + operation is UNION of indexes
d31 = d1+d2
# indexs not present in the other dataframe get a NaN
# do you can obvs remove nan values like before
d3.dropna().style.highlight_null(null_color='red')

,A,B
2022-01-03 00:00:00,0.672436,24.000000
2022-01-04 00:00:00,0.854795,24.000000
2022-01-05 00:00:00,0.624787,24.000000
2022-01-06 00:00:00,1.172271,9.000000
2022-01-07 00:00:00,0.798002,11.000000
2022-01-08 00:00:00,1.037257,43.000000
2022-01-09 00:00:00,1.310482,11.000000
2022-01-10 00:00:00,1.387156,26.000000


## df.align()

Pandas [align](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html) works to join the dataframes in interesting ways.

In [70]:
# align returns two dataframes - left and right as a result of a join
# default join is outer - the union of indices will be used as index for left and right
left, right = d1.align(d2, join='outer')
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.821870,13.000000
2022-01-02 00:00:00,0.161661,18.000000
2022-01-03 00:00:00,0.515014,7.000000
2022-01-04 00:00:00,0.854069,16.000000
2022-01-05 00:00:00,0.346648,9.000000
2022-01-06 00:00:00,0.835336,1.000000
2022-01-07 00:00:00,0.326867,7.000000
2022-01-08 00:00:00,0.756280,23.000000
2022-01-09 00:00:00,0.828364,1.000000
2022-01-10 00:00:00,0.615816,17.000000


In [71]:
# inner join
left, right = d1.align(d2, join='inner')
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.821870,13
2022-01-02 00:00:00,0.161661,18
2022-01-03 00:00:00,0.515014,7
2022-01-04 00:00:00,0.854069,16
2022-01-05 00:00:00,0.346648,9
2022-01-06 00:00:00,0.835336,1
2022-01-07 00:00:00,0.326867,7
2022-01-08 00:00:00,0.756280,23
2022-01-09 00:00:00,0.828364,1
2022-01-10 00:00:00,0.615816,17


In [72]:
# we can fill the NaNs with a certain value if needed
# let's fill with -100 so we can see it clearly
left, right = d1.align(d2, join='outer', fill_value = -100) 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.821870,13
2022-01-02 00:00:00,0.161661,18
2022-01-03 00:00:00,0.515014,7
2022-01-04 00:00:00,0.854069,16
2022-01-05 00:00:00,0.346648,9
2022-01-06 00:00:00,0.835336,1
2022-01-07 00:00:00,0.326867,7
2022-01-08 00:00:00,0.756280,23
2022-01-09 00:00:00,0.828364,1
2022-01-10 00:00:00,0.615816,17


In [73]:
# left join: left index is added to right, but not the other way
left, right = d1.align(d2, join='left') 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.821870,13
2022-01-02 00:00:00,0.161661,18
2022-01-03 00:00:00,0.515014,7
2022-01-04 00:00:00,0.854069,16
2022-01-05 00:00:00,0.346648,9
2022-01-06 00:00:00,0.835336,1
2022-01-07 00:00:00,0.326867,7
2022-01-08 00:00:00,0.756280,23
2022-01-09 00:00:00,0.828364,1
2022-01-10 00:00:00,0.615816,17


In [74]:
# right join: right index is added to left, but not the other way
left, right = d1.align(d2, join='right') 
render_df_side_by_side(left, right)

,A,B
2022-01-03 00:00:00,0.515014,7.000000
2022-01-04 00:00:00,0.854069,16.000000
2022-01-05 00:00:00,0.346648,9.000000
2022-01-06 00:00:00,0.835336,1.000000
2022-01-07 00:00:00,0.326867,7.000000
2022-01-08 00:00:00,0.756280,23.000000
2022-01-09 00:00:00,0.828364,1.000000
2022-01-10 00:00:00,0.615816,17.000000
2022-01-11 00:00:00,nan,nan
2022-01-12 00:00:00,nan,nan


There's more to explore in align, and it can get confusing, so beginners be careful, take time and try to work out the result before you execute to build intuition.

# Operations on data

## Stats

Operations in general *exclude* missing data

In [75]:
# arithmetic mean, for each column (axis = 0)
df.mean()

A     0.064782
B    -0.232884
C    -0.009091
D     0.267528
F    15.500000
dtype: float64

In [76]:
# mean across a row (axis = 1)
df.mean(1)

2022-06-19    2.414546
2022-06-20    1.314342
2022-06-21    2.381528
2022-06-22    2.637028
2022-06-23    3.636234
2022-06-24    2.999276
2022-06-25    3.649624
2022-06-26    4.329346
2022-06-27    3.507630
2022-06-28    4.311116
Freq: D, dtype: float64

## Apply

In [77]:
# apply custom lambdas
df.apply(lambda x: x.max()-x.min())

A    2.664674
B    3.705025
C    2.733648
D    2.533202
F    9.000000
dtype: float64

In [78]:
df

,A,B,C,D,F
2022-06-19,1.525287,-1.583237,0.131429,0.999250,11
2022-06-20,-1.139387,-2.549099,-0.523917,-1.215889,12
2022-06-21,-0.024668,-1.885221,0.434181,0.383347,13
2022-06-22,0.046389,0.103630,-0.387595,-0.577285,14
2022-06-23,0.681873,1.155926,1.646095,-0.302723,15
2022-06-24,0.252842,-0.594838,-0.827767,0.166141,16
2022-06-25,-0.536672,1.097415,0.257771,0.429608,17
2022-06-26,0.259723,1.056366,1.013331,1.317313,18
2022-06-27,-0.912890,0.185351,-1.087552,0.353240,19
2022-06-28,0.495321,0.684865,-0.746881,1.122277,20


In [79]:
# each subsequent value is a sum of all values before it in the respective column
cumsum_df = df.apply(np.cumsum)
render_df_side_by_side(df, cumsum_df, "OG Dataframe", "Cumulative Sum")

,A,B,C,D,F
2022-06-19 00:00:00,1.525287,-1.583237,0.131429,0.999250,11
2022-06-20 00:00:00,-1.139387,-2.549099,-0.523917,-1.215889,12
2022-06-21 00:00:00,-0.024668,-1.885221,0.434181,0.383347,13
2022-06-22 00:00:00,0.046389,0.103630,-0.387595,-0.577285,14
2022-06-23 00:00:00,0.681873,1.155926,1.646095,-0.302723,15
2022-06-24 00:00:00,0.252842,-0.594838,-0.827767,0.166141,16
2022-06-25 00:00:00,-0.536672,1.097415,0.257771,0.429608,17
2022-06-26 00:00:00,0.259723,1.056366,1.013331,1.317313,18
2022-06-27 00:00:00,-0.912890,0.185351,-1.087552,0.353240,19
2022-06-28 00:00:00,0.495321,0.684865,-0.746881,1.122277,20


## Histogramming

Frequencies, 'nuff said

```value_counts()```

In [80]:
series1 = pd.Series(np.random.randint(0,5, size = 25))
series1.value_counts()

3    7
2    6
1    5
0    4
4    3
dtype: int64

## String Methods

In [81]:
# build a series
some_string = "SERIes and Index are EqUIppeD WITh A seT Of stRINg PROCESSING METHoDs In tHe ```sTR``` attrIBute THAT MakE IT eASy To oPerATE ON EACh ELEmeNt OF The aRrAY. NOtE thAt PAtTERN-maTching in ```sTR``` gENeralLY UsES reGUlAr eXpreSsiONs bY DEfault (aND In Some cases AlwayS uSEs tHeM)."
str_series = pd.Series(some_string.split()) # split splits on whitespace by default

In [82]:
# manipulating the case - lower, upper etc.
# also the length of each string, split, replace, yada yada...
# very important when cleaning data (column names may have stupid whitespace, bad case, spacing etc.)
low = str_series.str.lower()
up = str_series.str.upper()

low_up_df = pd.DataFrame(
    {
        'og': str_series,
        'length': str_series.str.len(),
        'low': low,
        'up': up,
        'split_low_on_i': low.str.split('i', expand = False), # expand = True will break this usecase, try elsewhere
        'replace_a_with_star_in_low': low.str.replace('a','*', case=False, regex=True) # yeah, you can mess with regular expns
    })

low_up_df.tail(10)

,og,length,low,up,split_low_on_i,replace_a_with_star_in_low
36,eXpreSsiONs,11,expressions,EXPRESSIONS,"[express, ons]",expressions
37,bY,2,by,BY,[by],by
38,DEfault,7,default,DEFAULT,[default],def*ult
39,(aND,4,(and,(AND,[(and],(*nd
40,In,2,in,IN,"[, n]",in
41,Some,4,some,SOME,[some],some
42,cases,5,cases,CASES,[cases],c*ses
43,AlwayS,6,always,ALWAYS,[always],*lw*ys
44,uSEs,4,uses,USES,[uses],uses
45,tHeM).,6,them).,THEM).,[them).],them).


See [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#text-string-methods) for more string operations.

# Merge

Combine series and dataframes, use set logic for indexes and relational algebra for joins/merges

## Concat

Chill, we've done this before. The same join = inner/outer/left/right drill. 

In [83]:
# let's make some more dataframes
concatDF = pd.DataFrame(np.random.randn(10,4)) # 10 rows and 4 columns of random numbers
pieces = [concatDF[:3], concatDF[3:7], concatDF[7:]]
reconstructedDF = pd.concat(pieces)
display('broken DFs: ',pieces)
render_df_side_by_side(concatDF, reconstructedDF, "OG", "Concatenated")

'broken DFs: '

[          0         1         2         3
 0  0.509208  0.643084  0.623442  1.192363
 1 -2.177861 -0.714229 -0.471440  1.893455
 2  1.599285  0.388721  0.629479  0.204820,
           0         1         2         3
 3 -0.656701 -0.729735 -0.196392 -1.231606
 4  0.688394  0.772395 -0.120837 -0.008587
 5  0.487108 -1.524733 -0.691213  0.805689
 6 -1.002281 -0.515187  3.350761  1.224586,
           0         1         2         3
 7 -1.046476  0.225661 -0.315999  0.896613
 8  1.976262 -0.945545  1.455855  1.553661
 9  0.727573  0.891358  0.436657 -0.562880]

,0,1,2,3
0,0.509208,0.643084,0.623442,1.192363
1,-2.177861,-0.714229,-0.471440,1.893455
2,1.599285,0.388721,0.629479,0.204820
3,-0.656701,-0.729735,-0.196392,-1.231606
4,0.688394,0.772395,-0.120837,-0.008587
5,0.487108,-1.524733,-0.691213,0.805689
6,-1.002281,-0.515187,3.350761,1.224586
7,-1.046476,0.225661,-0.315999,0.896613
8,1.976262,-0.945545,1.455855,1.553661
9,0.727573,0.891358,0.436657,-0.562880


_From the [page](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html#concat)_:
Adding a column to a DataFrame is relatively fast. However, adding a row requires a copy, and may be expensive. We recommend passing a pre-built list of records to the DataFrame constructor instead of building a DataFrame by iteratively appending records to it.

## Join
SQL style merges. 

In [84]:
# two data frames
sl = pd.Series(np.random.randint(10, size=6))
sr = sl.shift(2).fillna(0).astype(int)
left = pd.DataFrame(
    {
        'key': list('ABCDEF'),
        'lval': sl
    }
)

right = pd.DataFrame(
    {
        'key': list('ABBCDE'),
        'rval': sr
    }
)

In [85]:
render_df_side_by_side(left, right, 'left', 'right')

,key,lval
0,A,2
1,B,1
2,C,2
3,D,9
4,E,7
5,F,4
,key,rval
0,A,0
1,B,0
2,B,2


In [86]:
result = pd.merge(left, right, on='key')
result

,key,lval,rval
0,A,2,0
1,B,1,0
2,B,1,2
3,C,2,1
4,D,9,2
5,E,7,9


# Grouping

"Group By" really means one or more of the following happening:
1. **Split**: data is broken into groups based on some criteria
2. **Apply**: a data operation is executed on each of those groups
3. **Combine**: the results are combined back into a data structure

In [87]:
grp_df = pd.DataFrame(
    {
        'colA': list('AABCAABDAA'),
        'colB': list('1234567890'),
        'colC': np.random.randn(10),
        'colD': np.random.randn(10)
    }
)
grp_df

,colA,colB,colC,colD
0,A,1,-1.566336,0.081260
1,A,2,0.821343,1.562684
2,B,3,1.317486,-0.574257
3,C,4,0.896743,0.442754
4,A,5,1.059647,-0.308096
5,A,6,-0.423634,0.420111
6,B,7,-0.077484,-1.096573
7,D,8,1.354606,0.164838
8,A,9,0.240491,0.459790
9,A,0,0.316000,-0.473361


In [88]:
# a DataFrameGroupBy object object
grpby = grp_df.groupby('colA')

# list all the groups found
g = grpby.groups
g

{'A': [0, 1, 4, 5, 8, 9], 'B': [2, 6], 'C': [3], 'D': [7]}

In [89]:
# to create a dataframe from the group use something like
grpby.get_group('A')

,colA,colB,colC,colD
0,A,1,-1.566336,0.081260
1,A,2,0.821343,1.562684
4,A,5,1.059647,-0.308096
5,A,6,-0.423634,0.420111
8,A,9,0.240491,0.459790
9,A,0,0.316000,-0.473361


### Find all the groups

In [94]:
# but we may not know all the groups in the dataframe.
# so got to find a way to list all groups as dataframes.
# 
# some basic python helps create a list of groupby names
grpby_labels_list = [*g.keys()] # using the unpacking operator *
# alternatively
# grpby_labels_list = list(g.keys())
grpby_labels_list

['A', 'B', 'C', 'D']

In [91]:
# create a dict where the group label is the key and the group dataframe is the value
grps = {lbl: grpby.get_group(lbl) for lbl in grpby_labels_list}

# now use the dict to access individual dataframes created out of the groupby operation
grps['B']

# question: can we create a dataframe of dataframes?

,colA,colB,colC,colD
2,B,3,1.317486,-0.574257
6,B,7,-0.077484,-1.096573


In [92]:
# You can see the datatype of the value is a pandas dataframe
type(grps['A'])

pandas.core.frame.DataFrame

In [93]:
# get a sum of values from each group
grp_df.groupby('colA').sum()

,colC,colD
colA,,
A,0.447511,1.742388
B,1.240003,-1.670830
C,0.896743,0.442754
D,1.354606,0.164838
